<a href="https://colab.research.google.com/github/ttogle918/NLU_3-/blob/main/%EC%B5%9C%EC%A7%80%ED%98%84_sts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLU - 문장 유사도 계산 (STS)**



- 과제 목표
  - 두 개의 한국어 문장을 입력받아 두 문장의 의미적 유사도를 출력
  - regression task ( 0 <= target <= 5 )
- 학습 데이터 셋 ( 다운로드 가능 & 제공 예정 )
  - KLUE-STS
    - AIRBNB ( 리뷰 )
    - policy ( 뉴스 )
    - paraKOQC ( 스마트홈 쿼리 )
- 과제 결과물
  - 학습된 모델 ( 모델 자유 선택 ) ( train set만 사용해 학습 )
  - 학습 방식 보고서
    - 어떤 모델을 선택했나
    - 어떻게 파라미터를 튜닝했나
    - 어떤 훈련 과정을 거쳤는가
  - dev set score ( F1 )
  - 문장 유사도를 출력하는 API ( 프레임워크 자유 선택 )

- [graykode/ALBERT-Pytorch](https://github.com/graykode/ALBERT-Pytorch)
- [huggingface](https://huggingface.co/docs/transformers/model_doc/albert)


유사도 계산.. 순서는 상관없는 것 같다..

albert가 sop(문장 순서 예측)을 통해 모델을 훈련하기 때문에 사용하려고 했는데, 유사도만 계산하는 것이기 때문에 albert를 사용할 때 장점이 크지는 않을 것 같다.

해야할 부분 

1. train data에서 train할 동안 검증할 validation 추출?
2. 모델 logit 0~5 사이로 정규화
3. 모델 클래스 작성!
4. train 메소드 작성..
5. ...

In [ ]:
# model load
!pip install pytorch-transformers

In [ ]:
!pip install transformers
!pip install datasets

In [3]:
import torch
from transformers import BertForNextSentencePrediction, BertTokenizerFast, BertConfig

- "klue/roberta-large"
- "klue/roberta-small"
- "klue/roberta-base"
- "klue/bert-base"
- [klue에 등록된 모델](https://huggingface.co/klue)
- [한국어언어모델](https://littlefoxdiary.tistory.com/81)

In [45]:
# https://huggingface.co/kykim/albert-kor-base

tokenizer = BertTokenizerFast.from_pretrained("snunlp/KR-Medium")

# KLUE 데이터셋

[klue-sts-벤치마크-구조-보기](https://velog.io/@soyoun9798/KLUE-STS-%EB%B2%A4%EC%B9%98%EB%A7%88%ED%81%AC-%EA%B5%AC%EC%A1%B0-%EB%B3%B4%EA%B8%B0)


In [6]:
from datasets import load_dataset
dataset = load_dataset('klue', 'sts')

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
print(f"type(dataset) : {type(dataset)}")
print(f"key : {dataset.keys()}")
print(f"type dataset[train] : {type(dataset['train'])}")
print(f"dataset[train] : {dataset['train']} \n\n")
# labels : { 이진분류 : 1, 반올림 값 : 3.7, 실제 label 값 : 3.71422... }
dataset['train'][0]

type(dataset) : <class 'datasets.dataset_dict.DatasetDict'>
key : dict_keys(['train', 'validation'])
type dataset[train] : <class 'datasets.arrow_dataset.Dataset'>
dataset[train] : Dataset({
    features: ['guid', 'source', 'sentence1', 'sentence2', 'labels'],
    num_rows: 11668
}) 




{'guid': 'klue-sts-v1_train_00000',
 'labels': {'binary-label': 1, 'label': 3.7, 'real-label': 3.714285714285714},
 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
 'source': 'airbnb-rtt'}

In [43]:
# 데이터 10개만 확인
i = 0
for d in dataset['train'] :
  if i == 10 : break
  print(d)
  i += 1

{'guid': 'klue-sts-v1_train_00000', 'source': 'airbnb-rtt', 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.', 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.', 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}
{'guid': 'klue-sts-v1_train_00001', 'source': 'policy-sampled', 'sentence1': '위반행위 조사 등을 거부·방해·기피한 자는 500만원 이하 과태료 부과 대상이다.', 'sentence2': '시민들 스스로 자발적인 예방 노력을\xa0한 것은 아산 뿐만이 아니었다.', 'labels': {'label': 0.0, 'real-label': 0.0, 'binary-label': 0}}
{'guid': 'klue-sts-v1_train_00002', 'source': 'paraKQC-sampled', 'sentence1': '회사가 보낸 메일은 이 지메일이 아니라 다른 지메일 계정으로 전달해줘.', 'sentence2': '사람들이 주로 네이버 메일을 쓰는 이유를 알려줘', 'labels': {'label': 0.3, 'real-label': 0.3333333333333333, 'binary-label': 0}}
{'guid': 'klue-sts-v1_train_00003', 'source': 'policy-sampled', 'sentence1': '긴급 고용안정지원금은 지역고용대응 등 특별지원금, 지자체별 소상공인 지원사업, 취업성공패키지, 청년구직활동지원금, 긴급복지지원제도 지원금과는 중복 수급이 불가능하다.', 'sentence2': '고용보험이 1차 고용안전망이라면, 국민취업지원제도는 2차 고용안전망입니다.', 'labels': {'label': 0.6, 'real-la

In [35]:
# tokenizer 잘 되는지 확인
tensorized_input = tokenizer(
        '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
 '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
tensorized_input

{'input_ids': tensor([[    2, 23903, 27093, 19611, 17331, 17744, 18404, 21658,  8052, 23903,
         13992,  2016,     3, 22668, 15996,  8107, 27093, 14456, 19381,  5099,
         14204, 18404, 18210, 21658,  8052, 22668, 15996, 13992,  2016,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

# Dataset Tokenizing -> dataLoader

In [9]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [36]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    def __init__(self, dataset):
        self.sentence1, self.sentence2, self.labels = self.make_dataset(dataset)
    
    def make_dataset(self, dataset):
        """
        self.label : dataset의 label의 list
        self.input : sentence1, sentence2를 tokenizer한 값을 이어 붙임 
        """
        sentence1, sentence2 = [], []
        rlabels = []  # real-label
        # blabel = [] # binary-label

        for data in dataset :
          rlabels.append(data['labels']['real-label'])
          # blabel.append(data['labels']['binary-label'])
          sentence1.append(data['sentence1'])
          sentence2.append(data['sentence2'])
        return sentence1, sentence2, rlabels
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sentence1[idx], self.sentence2[idx], self.labels[idx]

In [37]:
train_dataset = CustomDataset(dataset['train'])
valid_dataset = CustomDataset(dataset['validation'])

In [38]:
def custom_collate_fn(batch):
    """
    - batch: list of tuples (input1_data(string), input2_data(string), target_data(int))
    
    한 배치 내 문장들을 tokenizing 한 후 텐서로 변환함. 
    이때, dynamic padding (즉, 같은 배치 내 토큰의 개수가 동일할 수 있도록, 부족한 문장에 [PAD] 토큰을 추가하는 작업)을 적용
    
    한 배치 내 레이블(target)은 텐서화 함.
    
    (input, target) 튜플 형태를 반환.
    """
    input1_list, input2_list, target_list = [], [], []
    
    for _input1, _input2, _target in batch:
        input1_list.append(_input1)
        input2_list.append(_input2)
        target_list.append(_target)
    
    tensorized_input = tokenizer(
        input1_list, input2_list,
        add_special_tokens=True,
        padding="longest",  # 배치내 가장 긴 문장을 기준으로 부족한 문장은 [PAD] 토큰을 추가
        truncation=True, # max_length를 넘는 문장은 이 후 토큰을 제거함
        max_length=512,
        return_tensors='pt' # 토크나이즈된 결과 값을 텐서 형태로 반환
    )
    
    tensorized_label = torch.tensor(target_list)
    
    return tensorized_input, tensorized_label

In [39]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size =32,
    sampler = RandomSampler(train_dataset),
    collate_fn = custom_collate_fn
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size =64,
    sampler = SequentialSampler(valid_dataset),
    collate_fn = custom_collate_fn
)

In [41]:
# tokenizing 잘 되었는지 확인
for input_batch, target_batch in train_dataloader:
    print(f"Batch input shape: {input_batch['input_ids'].shape}")
    print(input_batch['input_ids'])
    print(f"Batch target shape: {target_batch.shape}")
    break

Batch input shape: torch.Size([32, 99])
tensor([[    2, 19943,  8144,  ...,     0,     0,     0],
        [    2, 19812, 23903,  ...,     0,     0,     0],
        [    2,  6035, 37424,  ...,     0,     0,     0],
        ...,
        [    2, 14254, 19926,  ..., 14015,  2016,     3],
        [    2, 23380, 18146,  ...,     0,     0,     0],
        [    2, 14098, 36629,  ...,     0,     0,     0]])
Batch target shape: torch.Size([32])


In [ ]:
class Config(BertConfig):
  def __init__(self) :
    super().__init__()
    self.task= 'sts'
    self.bert_model = 'snunlp/KR-Medium'
    self.max_len= 512
    self.dropout_rate= 0.1

In [ ]:
# 모델 클래스
class CustomSTS(nn.Module):
    def __init__(self, hidden_size: int, n_label: int, config):
        super(CustomSTS, self).__init__()
        self.config = config
        self.model = BertForNextSentencePrediction.from_pretrained(self.config.bert_model)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )
        
        logits = self.model(input_ids, token_type_ids=token_type_ids).logits
        probs = self.softmax(logits)    # 0~5사이로 고쳐야함!
        return probs

# train

In [42]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

In [ ]:
# def train(model, data_loader):

#         # 배치 단위 평균 loss와 총 평균 loss 계산하기위해 변수 생성
#         total_loss, batch_loss, batch_count = 0,0,0
        
#         # model을 train 모드로 설정 & device 할당
#         model.train()
#         model.to(device)
        
#         # data iterator를 돌면서 하나씩 학습
#         for step, batch in enumerate(data_loader):
#             batch_count+=1
            
#             # tensor 연산 전, 각 tensor에 device 할당
#             batch = tuple(item.to(device) for item in batch)
            
#             batch_input, batch_label = batch
            
#             # batch마다 모델이 갖고 있는 기존 gradient를 초기화
#             model.zero_grad()
            
#             # forward
#             logits = model(**batch_input)
            
#             # loss
#             loss = loss_fct(logits, batch_label)
#             batch_loss += loss.item()
#             total_loss += loss.item()
            
#             # backward -> 파라미터의 미분(gradient)를 자동으로 계산
#             loss.backward()
            
#             # optimizer 업데이트
#             optimizer.step()
            
#             # 배치 10개씩 처리할 때마다 평균 loss를 출력
#             if (step % 10 == 0 and step != 0):
#                 print(f"Step : {step}, Avg Loss : {batch_loss / batch_count:.4f}")
                
#                 # 변수 초기화 
#                 batch_loss, batch_count = 0,0
        
#         print(f"Mean Loss : {total_loss/(step+1):.4f}")
#         print("Train Finished")